# Set Up

In [71]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
from flask import Flask, request, jsonify
import requests

# Get Data

**Query what data user needs**

In [72]:
user_needs = {
  "id_user": "1",
  "genre": "Pop"
}

In [73]:
def get_events(genre):
    url = f'https://elise-project.mdwisu.shop/events/getEventForMl?genre={genre}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['data'])
    return df

In [74]:
df = get_events(user_needs['genre'])

In [75]:
df

,id_user,id_event,nama_event,harga_tiket,durasi,genre,nama_musisi,id_event_organizer
0,4,6,Popex Musicals,500000,6,Pop,Bunga Citra Lestari,4
1,4,25,Cello Celebration,470000,8,Pop,Andra and The Backbone,4
2,4,26,SuperSonics,300000,8,Pop,Vierra,4


In [76]:
music_genres = {
    'pop': 4,
    'jazz': 3,
    'dangdut': 0,
    'electronic': 1,
    'r&b': 5,
    'rock': 6,
    'indie': 2
}

In [77]:
genre = music_genres[user_needs['genre'].lower()]
genre

4

In [78]:
df['genre'] = genre

In [79]:
df['id_user'] = user_needs['id_user']

In [80]:
df

,id_user,id_event,nama_event,harga_tiket,durasi,genre,nama_musisi,id_event_organizer
0,1,6,Popex Musicals,500000,6,4,Bunga Citra Lestari,4
1,1,25,Cello Celebration,470000,8,4,Andra and The Backbone,4
2,1,26,SuperSonics,300000,8,4,Vierra,4


In [81]:
df.dtypes

id_user               object
id_event               int64
nama_event            object
harga_tiket            int64
durasi                 int64
genre                  int64
nama_musisi           object
id_event_organizer     int64
dtype: object

# Load the model

In [82]:
model = tf.keras.models.load_model('./content_model')

# Convert Df to Tensorflow Dataset

In [83]:
ratings = tf.data.Dataset.from_tensor_slices(
    dict(df[['id_user', 'id_event', 'nama_event', 'harga_tiket', 'durasi', 'genre', 'id_event_organizer']]))
    
ratings = ratings.map(lambda x: {
    'durasi': float(x['durasi']),
    'genre': int(x['genre']),
    'harga_tiket': float(x['harga_tiket']),
    'id_event': str(x['id_event']),
    'id_event_organizer': str(x['id_event_organizer']),
    'id_user': str(x['id_user']),
    'nama_event': x['nama_event'],
})

cached_test = ratings.batch(4096).cache()

# Prediction

In [84]:
prediction = model.predict(cached_test)

1/1 [==============================] - 1s 720ms/step


In [85]:
prediction

array([[3.0109935],
       [2.5304615],
       [3.2819686]], dtype=float32)

In [86]:
df['rating_prediction'] = pd.Series(prediction.flatten())

In [87]:
df

,id_user,id_event,nama_event,harga_tiket,durasi,genre,nama_musisi,id_event_organizer,rating_prediction
0,1,6,Popex Musicals,500000,6,4,Bunga Citra Lestari,4,3.010993
1,1,25,Cello Celebration,470000,8,4,Andra and The Backbone,4,2.530462
2,1,26,SuperSonics,300000,8,4,Vierra,4,3.281969


# Sort Descending by rating _prediction

In [91]:
df.sort_values(by='rating_prediction', ascending=False, inplace=True)

In [92]:
df

,id_user,id_event,nama_event,harga_tiket,durasi,genre,nama_musisi,id_event_organizer,rating_prediction
2,1,26,SuperSonics,300000,8,4,Vierra,4,3.281969
0,1,6,Popex Musicals,500000,6,4,Bunga Citra Lestari,4,3.010993
1,1,25,Cello Celebration,470000,8,4,Andra and The Backbone,4,2.530462


# Df to JSON

In [93]:
json_data= df.to_json(orient='records')

In [94]:
json_data

'[{"id_user":"1","id_event":26,"nama_event":"SuperSonics","harga_tiket":300000,"durasi":8,"genre":4,"nama_musisi":"Vierra","id_event_organizer":4,"rating_prediction":3.2819685936},{"id_user":"1","id_event":6,"nama_event":"Popex Musicals","harga_tiket":500000,"durasi":6,"genre":4,"nama_musisi":"Bunga Citra Lestari","id_event_organizer":4,"rating_prediction":3.0109934807},{"id_user":"1","id_event":25,"nama_event":"Cello Celebration","harga_tiket":470000,"durasi":8,"genre":4,"nama_musisi":"Andra and The Backbone","id_event_organizer":4,"rating_prediction":2.5304615498}]'